In [1]:
!pip install -U tensorflow==2.17 #2.15 17 - wehere wixed tf lite coverter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 2.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1


In [3]:
import numpy as np
import pandas as pd
from glob import glob
import os
import pathlib
import tensorflow as tf
import keras
import seaborn as sns

#models 
from tensorflow.keras.applications import EfficientNetB1, EfficientNetB3, EfficientNetB5

#preproces
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

from PIL import Image
import cv2 as cv
import matplotlib.pyplot as plt


from sklearn.metrics import confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns


from collections import Counter
# for proper loading img
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True



In [4]:
tf.__version__,

('2.17.0',)

## Data Load

In [5]:
SIZE = 320 #300
BS = 32 #16
imgs = glob(r'/kaggle/input/food41/images/*/*')
labels = [el.split('/')[5] for el in imgs ]

X_train, X_test, y_train, y_test = train_test_split(imgs, labels, test_size=0.5, random_state=11, stratify=labels)
train = pd.DataFrame(list(zip(X_train,y_train)), columns =['X','y'])
test = pd.DataFrame(list(zip(X_test,y_test)), columns =['X','y'])

datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.15,
        zoom_range=0.15,)
valid_gen = ImageDataGenerator(rescale=1./255)



train_gen = datagen.flow_from_dataframe(train, x_col='X', y_col="y", target_size=(SIZE, SIZE), shuffle=True)
test_gen = valid_gen.flow_from_dataframe(test, x_col='X', y_col="y", target_size=(SIZE, SIZE), shuffle=False)



Found 50500 validated image filenames belonging to 101 classes.
Found 50500 validated image filenames belonging to 101 classes.


In [15]:
model = tf.keras.models.load_model('/kaggle/input/weights_food101_efb5_e20/tensorflow2/default/1/food101_efb3_p25.keras')

## Train

the model is efb5 (but in the name and discreption may occurs other name)

In [ ]:
loss = tf.keras.losses.CategoricalFocalCrossentropy(gamma=6, from_logits=False)
learning_rate = 2.1600e-04
optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)   
model.compile(loss=loss, metrics=['accuracy'], optimizer=optimizer)



early_stop = EarlyStopping(monitor='val_loss', verbose=1, patience=3, restore_best_weights=True, start_from_epoch=5)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=2, min_lr=1.0000e-08) #0
epoch = 10
history = model.fit(train_gen, validation_data=test_gen, epochs=epoch, callbacks=[early_stop, reduce_lr],  batch_size = BS )

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1579/1579 ━━━━━━━━━━━━━━━━━━━━ 2807s 2s/step - accuracy: 0.6733 - loss: 3.2735 - val_accuracy: 0.1012 - val_loss: 4.2427 - learning_rate: 2.1600e-04
Epoch 2/10
1579/1579 ━━━━━━━━━━━━━━━━━━━━ 2102s 1s/step - accuracy: 0.7115 - loss: 3.1126 - val_accuracy: 0.3479 - val_loss: 3.7049 - learning_rate: 2.1600e-04
Epoch 3/10
1579/1579 ━━━━━━━━━━━━━━━━━━━━ 2101s 1s/step - accuracy: 0.7192 - loss: 3.1049 - val_accuracy: 0.3633 - val_loss: 3.7444 - learning_rate: 2.1600e-04
Epoch 4/10
1579/1579 ━━━━━━━━━━━━━━━━━━━━ 2103s 1s/step - accuracy: 0.7288 - loss: 3.0971 - val_accuracy: 0.5289 - val_loss: 3.4246 - learning_rate: 2.1600e-04
Epoch 5/10
1579/1579 ━━━━━━━━━━━━━━━━━━━━ 2160s 1s/step - accuracy: 0.7352 - loss: 3.0893 - val_accuracy: 0.4380 - val_loss: 3.6758 - learning_rate: 2.1600e-04
Epoch 6/10
1579/1579 ━━━━━━━━━━━━━━━━━━━━ 2103s 1s/step - accuracy: 0.7479 - loss: 3.0840 - val_accuracy: 0.4416 - val_loss: 3.5233 - learning_rate: 2.1600e-04
Epoch 7/10
1579/1579 ━━━━━━━━━━━━━━━━━━━━ 2105s 1s/

In [5]:
model.save('food101_efb5_p30.keras')

In [ ]:
model.save('food101_efb5_p30.h5')

In [6]:
model.save_weights('weights_food101_efb5_e30.weights.h5')


## Metrics

In [ ]:
val = model.evaluate(test_gen)
test = model.evaluate(train_gen)

In [ ]:
def show_history(history):
    plt.figure(figsize=(18,8))

    plt.suptitle('Loss and Accuracy Plots', fontsize=18)

    plt.subplot(1,2,1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.xlabel('Number of epochs', fontsize=15)
    plt.ylabel('Loss', fontsize=15)

    plt.subplot(1,2,2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.xlabel('Number of epochs', fontsize=14)
    plt.ylabel('Accuracy', fontsize=14)
    plt.show()

show_history(history)

In [ ]:
train_gen = datagen.flow_from_dataframe(train, x_col='X', y_col="y", target_size=(SIZE, SIZE), shuffle=False)
test_gen = valid_gen.flow_from_dataframe(test, x_col='X', y_col="y", target_size=(SIZE, SIZE), shuffle=False)


In [ ]:
pred_test = model.predict(test_gen)
pred_train = model.predict(train_gen)

In [ ]:
def report(true_labels, pred_labels, gen, n_clases=101):
    
    with pd.option_context("future.no_silent_downcasting", True):
        true_labels = true_labels.replace(gen.class_indices).infer_objects(copy=False)
    true_labels = keras.utils.to_categorical(true_labels, num_classes=n_clases)


    predicted_classes = np.argmax(pred_labels, axis=1)
    true_classes = np.argmax(true_labels, axis=1)

    print(classification_report(true_classes, predicted_classes))

    cm = confusion_matrix(true_classes, predicted_classes)


    plt.figure(figsize=(101, 101))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()
    

## Report on test data

In [ ]:
report(test['y'], pred_test, test_gen)

## Report on train data

In [ ]:
report(train['y'], pred_train, train_gen)

## TF lite

In [16]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

Saved artifact at '/tmp/tmprbqmzdqe'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 320, 320, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 101), dtype=tf.float32, name=None)
Captures:
  134412531615696: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  134412146211856: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  134412976983904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134412976982672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134412976990592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134412976987248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134412976989360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134412976994464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134412976994640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134412976992176: TensorSpec(shape=(), dtype=tf.resource, name=Non

In [17]:
import pathlib
tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)

116685904